In [2]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier

def read(readdata):
    data = np.array(readdata);
    data = np.delete(data, 0, 0)
    data = np.swapaxes(data,0,1)
    for i in range(0, np.shape(data)[0]):
        data[i,:] = data[i,:] - np.min(data[i,:])
        data[i,:] = (data[i,:]/np.ndarray.max(data[i,:]))*2 - 1
    # Need to seperate the classification dimension:
    classIndex = np.shape(data)[0] - 1
    classification = data[classIndex]
    data = np.delete(data, classIndex, axis=0)
    data = data.astype(float)
    d = np.shape(data)[0]
    nPoints = np.shape(data)[1]
    return data,classification,d,nPoints
    '''
    colour = np.chararray((len(classification)),unicode = True)
    for i in range(len(classification)):
        if classification[i] == '1':
            colour[i] = 'r'
        elif classification[i] == '2':
            colour[i] = 'b'
        elif classification[i] == '3':
            colour[i] = 'y'
        elif classification[i] == '0':
            colour[i] = 'g'
   '''

In [7]:
def step(acoord,n,stepSize):
    acoord[n] = acoord[n] + stepSize
    return acoord

def getPolars(coord):
    '''polars = np.zeros((2,d))
    polars[1,0] = np.pi/2
    j = 1
    for i in range(d-1):
        polars[1,j] = coord[i]
        j+=1
    j = 0
    for i in range(d-1,2*d - 1):
        polars[0,j] = coord[i]
        j += 1
    return polars '''
    polars = np.zeros((2,d-1))
   
    for i in range(d-1):
        polars[0,i] = coord[i]
    j = 1
    polars[1,0] = np.pi/2
    for i in range(d-1,2*d - 3):
        polars[1,j] = coord[i]
        j += 1
    return polars 

def RotationMatrix(i, j, d, theta):
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R

def getWigner(polar):
    wig = RotationMatrix(0,1,d,polar[0])
    for i in range(1,d-1):
        wig = np.matmul(RotationMatrix(i,i+1,d,polar[i]),wig) 
    return wig
    
def getX(coord,d):
    polars = getPolars(coord)
    X = np.ones((2,d))
    X[0] = X[0]*0
    X[0,0] = 1
    i = 1
    for j in range(0,d):
        k = j 
        if j != d-1:
            X[i,j] = np.cos(polars[i,j])
        else:
            k = j-1
            X[i,j] = np.sin(polars[i,j-1])
        while k > 0:
            k -= 1
            X[i,j] = X[i,j]*np.sin(polars[i,k])
    wig = getWigner(polars[0])
    X[0] = wig.dot(X[0])
    X[1] = wig.dot(X[1])
    return X

def Transform(data, coord):
    nPoints = np.shape(data)[1]
    d = np.shape(data)[0]
    a = np.zeros((2,nPoints))
    X = getX(coord,d)  
    for p in range(0,nPoints):
        a[0,p] = np.dot(X[0],data[:,p])
        a[1,p] = np.dot(X[1],data[:,p])
    return a

def SVM(a):   
    xData = a[0]
    yData = a[1]
    nData = np.vstack((xData, yData)).T
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(nData, classification)
    score = clf.score(nData, classification)
    return score

def native(data):
    clf = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
    clf.fit(data.T, classification)
    score = clf.score(data.T, classification)
    return score

In [8]:
def findOptima(iterations, nSteps, data, costfunc):
    d = np.shape(data)[0]
    Optcoordlist = np.zeros((iterations,2*d-3))
    Optwlist = np.zeros(iterations)
    shufData = np.zeros((iterations,d,nPoints))
    Optw = 0
    stepSize = np.pi*2/nSteps
    for r in range(iterations):
        np.random.shuffle(data)
        coord = np.ones(2*d-3)*np.pi/2
        for j in range(2*d - 3): 
            n = 2*d - 4 - j
            a = Transform(data, coord)
            w = costfunc(a)
            singlescore = np.zeros(nSteps)
            tcoord = coord
            for i in range(0,nSteps):
                tcoord = step(tcoord,n,stepSize)
                a = Transform(data,tcoord)
                w = costfunc(a)
                singlescore[i] = w
                print(iterations*(2*d-3)*nSteps,r*(nSteps*(2*d-3)) + j*nSteps + i, end="\r")
            k = np.argmax(singlescore)
            coord[n] = coord[n] - np.pi*2
            coord = step(coord,n,(k+1)*stepSize)
            a = Transform(data, coord)
            w = costfunc(a)
        Optcoordlist[r] = coord
        Optwlist[r] = w
        shufData[r] = data
        l = np.argmax(Optwlist)
        Optw = Optwlist[l]
        Optcoord = Optcoordlist[l]
    return Optcoord, Optw, Optcoordlist, Optwlist, shufData

In [10]:
########READ in Data########

iterations = 5
resolution = 100


readdata = pd.read_csv("/Users/karanmukhi/Desktop/TheGrandTour/Datasets/wineData.txt", sep=",", header=None);
(data, classification, d, nPoints) = read(readdata)
print("wine",d,nPoints)
score = native(data)
print("Native score:", score)
(Optcoord, Optw, Optcoordlist, Optwlist) = findOptima(iterations, resolution, data, SVM)
print("wine max:", Optw)
np.savetxt("wineCoord.csv", Optcoord, delimiter=",")
print("")



wine 13 177
Native score: 0.9943502824858758


KeyboardInterrupt: 